In [ ]:
%matplotlib inline

In [ ]:
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
auth_data = {
    'grant_type'    : 'client_credentials',
    'client_id'     : '4840067a72b940a0bfb573681b84d320',
    'client_secret' : 'cff9770d549da86d5a6f539ac1df26fe03209610faa03ecbd57f0114fafc0389',
    'scope'         : 'read_product_data read_financial_data read_content'
}

# create Session instance
session = requests.Session()

# make a POST to retrieve access_token
auth_request = session.post('https://idfs.gs.com/as/token.oauth2', data = auth_data)
access_token_dict = json.loads(auth_request.text)
access_token = access_token_dict['access_token']

# update session headers
session.headers.update({'Authorization':'Bearer '+ access_token})

print(access_token)

In [ ]:
request_url = 'https://api.marquee.gs.com/v1/data/catalog/USCANFPP_MINI'
request = session.get(url=request_url)
data = json.loads(request.text)
print(data['fields'])

In [ ]:
import pandas as pd

def get_ids(num_ids):
    offset = 0
    gsids = []
    while len(gsids) < num_ids:
        request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/coverage?limit=100&offset={}'.format(offset)
        request = session.get(url=request_url)
        data = json.loads(request.text)
        df = pd.DataFrame(data['results'])
        if len(df) == 0:
            print('Reached limit {}'.format(len(gsids)))
            break
        offset += 100
        gsids += list(df.gsid)
        print('Saved', offset, 'ids')
    # Check that we aren't getting duplicates
    return gsids

gsids = get_ids(100000)

In [ ]:
def get_names(gsids):
    data = []
    for i in range(len(gsids) // 100):
        print(i)
        payload = {
            "where": {
                "gsid": gsids[100 * i: 100 * i + 100]
            },
            "fields": [ "gsid", "ticker", "name" ],
            "limit": 100
        }

        request_url = 'https://api.marquee.gs.com/v1/assets/data/query'
        request = session.post(url=request_url, json = payload)
        results = json.loads(request.text)
        data += results['results']
    return data
    
data = get_names(gsids)

In [ ]:
company_df = pd.DataFrame(data)
# company_df.to_csv('../data/processed/gs_companies_100k.csv', index=False)

In [129]:
scores = []

def get_2017_scores(company_gsid):
    payload = {
        "startDate": "2017-01-01",
        "endDate": "2017-12-31",
        "where": {
            "gsid": list(gsids)
        }
    }
    request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/query'
    request = session.post(url=request_url, json = payload)
    results = json.loads(request.text)
    return results['data']

get_2017_scores(company_df['gsid'][1])

[{'date': '2017-01-02',
  'financialReturnsScore': 0.452,
  'growthScore': 0.372,
  'gsid': '75154',
  'integratedScore': 0.514,
  'multipleScore': 0.286,
  'updateTime': '2017-05-11T19:47:36Z'},
 {'date': '2017-01-03',
  'financialReturnsScore': 0.454,
  'growthScore': 0.372,
  'gsid': '75154',
  'integratedScore': 0.514,
  'multipleScore': 0.286,
  'updateTime': '2017-05-11T19:47:36Z'},
 {'date': '2017-01-04',
  'financialReturnsScore': 0.454,
  'growthScore': 0.376,
  'gsid': '75154',
  'integratedScore': 0.516,
  'multipleScore': 0.28,
  'updateTime': '2017-05-11T19:47:36Z'},
 {'date': '2017-01-05',
  'financialReturnsScore': 0.452,
  'growthScore': 0.374,
  'gsid': '75154',
  'integratedScore': 0.514,
  'multipleScore': 0.286,
  'updateTime': '2017-05-11T19:47:36Z'},
 {'date': '2017-01-06',
  'financialReturnsScore': 0.452,
  'growthScore': 0.374,
  'gsid': '75154',
  'integratedScore': 0.508,
  'multipleScore': 0.302,
  'updateTime': '2017-05-11T19:47:36Z'},
 {'date': '2017-01-09

In [130]:
beer_tickers = [
    'ABEV',
    'BUD',
    'STZ',
    'DEO',
    'TAP',
    'CCU',
    'AKOA',
    'BREW',
    'BFA',
    'SAM',
    'STZ',
    'DEO',
    'VCO',
    'ROX',
    'BORN',
    'THST',
    'WVVI',
]

In [135]:
def get_2017_scores_ticker(ticker):
    payload = {
        "startDate": "2017-01-01",
        "endDate": "2017-12-31",
        "where": {
            "ticker": ticker,
        }
    }
    request_url = 'https://api.marquee.gs.com/v1/data/USCANFPP_MINI/query'
    request = session.post(url=request_url, json = payload)
    results = json.loads(request.text)
    return results['data']

scores = []
for ticker in beer_tickers:
    try:
        scores += get_2017_scores_ticker(ticker)
        print(ticker, 'in dataset')
    except KeyError:
        print(ticker, 'not in dataset')

ABEV not in dataset
BUD not in dataset
STZ in dataset
DEO not in dataset
TAP in dataset
CCU not in dataset
AKOA in dataset
BREW not in dataset
BFA not in dataset
SAM not in dataset
STZ in dataset
DEO not in dataset
VCO not in dataset
ROX not in dataset
BORN not in dataset
THST not in dataset
WVVI not in dataset


In [137]:
beer_score_df = pd.DataFrame(scores)

In [139]:
beer_score_df.to_csv('../data/processed/beer_scores.csv', index=False)

In [144]:
food_tickers = [
    'BWLD',
    'ARKR',
    'DENN',
    'NATH',
    'GTIM',
    'WEN',
    'CAKE',
    'WINGS',
    'RRGB',
    'PLKI',
    'PZZA',
    'GTIM',
]

food_scores = []
for ticker in food_tickers:
    try:
        food_scores += get_2017_scores_ticker(ticker)
        print(ticker, 'in dataset')
    except KeyError:
        print(ticker, 'not in dataset')
food_scores

BWLD in dataset
ARKR not in dataset
DENN not in dataset
NATH not in dataset
GTIM not in dataset
WEN not in dataset
CAKE not in dataset
WINGS in dataset
RRGB not in dataset
PLKI in dataset
PZZA not in dataset
GTIM not in dataset


[]

In [143]:
pd.DataFrame(food_scores).to_csv('../data/processed/food_scores.csv', index=False)